In [1]:
library(data.table)
library(qvalue)

In [2]:
## SLE (systemic lupus erythematosus) SMR (summarised Mendelian Randomisation) results

In [3]:
# Matrix eQTL results
matrix_eqtl_smr_results_dir = "/directflow/SCCGGroupShare/projects/angxue/proj/SAIGE-eQTL/SMR/"
matrix_ibd_file = paste0(matrix_eqtl_smr_results_dir, "sle_all_14_celltypes_all_smr.txt")
matrix_df = fread(matrix_ibd_file)
head(matrix_df,2)

probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,⋯,p_GWAS,b_eQTL,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_HEIDI,nsnp_HEIDI,Cell_type
<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
FCRL5,1,FCRL5,157483167,rs12741984,1,157534818,G,A,0.787234,⋯,0.9282,0.113780,0.00486738,7.500497e-121,-0.0500968,0.554584,0.9280233,0.7014684,20,BimmNaive
RPS8,1,RPS8,45240923,rs12120833,1,45241285,C,G,0.850097,⋯,0.4884,-0.105432,0.00500701,1.980368e-98,-0.5140770,0.742114,0.4884856,0.9091780,20,BimmNaive


In [4]:
# Calculate number of unique genes, and combination of genes + celltypes
length(unique(matrix_df$Gene))
matrix_df$comb = paste0(matrix_df$Gene,"_",matrix_df$Cell_type)
length(unique(matrix_df$comb))
head(matrix_df,2)

[1] 6088

[1] 14511

probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,⋯,b_eQTL,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_HEIDI,nsnp_HEIDI,Cell_type,comb
<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
FCRL5,1,FCRL5,157483167,rs12741984,1,157534818,G,A,0.787234,⋯,0.113780,0.00486738,7.500497e-121,-0.0500968,0.554584,0.9280233,0.7014684,20,BimmNaive,FCRL5_BimmNaive
RPS8,1,RPS8,45240923,rs12120833,1,45241285,C,G,0.850097,⋯,-0.105432,0.00500701,1.980368e-98,-0.5140770,0.742114,0.4884856,0.9091780,20,BimmNaive,RPS8_BimmNaive


In [5]:
nrow(matrix_df[matrix_df$p_SMR < 0.05/length(unique(matrix_df$Gene)),])
nrow(matrix_df[matrix_df$p_SMR < 0.05/length(unique(matrix_df$comb)),])

[1] 0

[1] 0

In [6]:
# SAIGE-QTL results
saige_eqtl_smr_results_dir = "/directflow/SCCGGroupShare/projects/angxue/proj/SAIGE-eQTL/SMR/saige_eQTL/"
saige_ibd_file = paste0(saige_eqtl_smr_results_dir, "sle_all_14_celltypes_all_smr.txt")
saige_df = fread(saige_ibd_file)
head(saige_df,2)

probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,⋯,p_GWAS,b_eQTL,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_HEIDI,nsnp_HEIDI,Cell_type
<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
NOC2L,1,NOC2L,879584,rs3748595,1,887560,A,C,0.0672147,⋯,0.14100,-0.153020,0.0352961,1.455500e-05,0.992681,0.712232,0.16338990,0.4676291,3,BimmNaive
C1orf86,1,C1orf86,2115903,rs11587831,1,2110848,T,G,0.7853000,⋯,0.06286,-0.115809,0.0148762,6.980209e-15,-1.081950,0.597518,0.07017984,0.9955862,8,BimmNaive


In [7]:
length(unique(saige_df$Gene))
saige_df$comb = paste0(saige_df$Gene,"_",saige_df$Cell_type)
length(unique(saige_df$comb))
head(saige_df,2)

[1] 5671

[1] 16413

probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,⋯,b_eQTL,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_HEIDI,nsnp_HEIDI,Cell_type,comb
<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
NOC2L,1,NOC2L,879584,rs3748595,1,887560,A,C,0.0672147,⋯,-0.153020,0.0352961,1.455500e-05,0.992681,0.712232,0.16338990,0.4676291,3,BimmNaive,NOC2L_BimmNaive
C1orf86,1,C1orf86,2115903,rs11587831,1,2110848,T,G,0.7853000,⋯,-0.115809,0.0148762,6.980209e-15,-1.081950,0.597518,0.07017984,0.9955862,8,BimmNaive,C1orf86_BimmNaive


In [8]:
# significant results at p<0.05/M with M either # genes or # gene-celltype combinations
nrow(saige_df[saige_df$p_SMR < 0.05/length(unique(saige_df$Gene)),])
nrow(saige_df[saige_df$p_SMR < 0.05/length(unique(saige_df$comb)),])

[1] 1

[1] 0

In [9]:
### p < 0.05/M with M-number of unique gene-celltype combinations tested by both methods

In [10]:
matrix_combs = unique(matrix_df$comb)
saige_combs = unique(saige_df$comb)
common_combs = saige_combs[saige_combs %in% matrix_combs]
length(common_combs)

[1] 11980

In [11]:
matrix_df_common = matrix_df[matrix_df$comb %in% common_combs,]
nrow(matrix_df_common)
nrow(matrix_df_common[matrix_df_common$p_SMR < (0.05/length(common_combs)),])

[1] 11980

[1] 0

In [12]:
saige_df_common = saige_df[saige_df$comb %in% common_combs,]
nrow(saige_df_common)
nrow(saige_df_common[saige_df_common$p_SMR < (0.05/length(common_combs)),])

[1] 11980

[1] 1

In [13]:
# significant results at FDR<5% instead (using qvalue)
matrix_df_common$qv = qvalue(matrix_df_common$p_SMR)$qvalues
nrow(matrix_df_common[matrix_df_common$qv<0.05,])

[1] 0

In [14]:
saige_df_common$qv = qvalue(saige_df_common$p_SMR)$qvalues
nrow(saige_df_common[saige_df_common$qv<0.05,])

[1] 21

In [16]:
# Using p < (0.05 / n_tests, gene-celltype comb)

matrix_sign_combs = matrix_df_common[matrix_df_common$p_SMR < (0.05/length(common_combs)),"comb"]$comb
length(matrix_sign_combs)
head(matrix_sign_combs)

saige_sign_combs = saige_df_common[saige_df_common$p_SMR < (0.05/length(common_combs)),"comb"]$comb
length(saige_sign_combs)
head(saige_sign_combs)

length(saige_sign_combs[saige_sign_combs %in% matrix_sign_combs])

[1] 0

character(0)

[1] 1

[1] "HLA-DRB1_NKact"

[1] 0

In [17]:
# Using FDR < 0.05

matrix_sign_combs = matrix_df_common[matrix_df_common$qv < 0.05,"comb"]$comb
length(matrix_sign_combs)

saige_sign_combs = saige_df_common[saige_df_common$qv < 0.05,"comb"]$comb
length(saige_sign_combs)

length(saige_sign_combs[saige_sign_combs %in% matrix_sign_combs])

[1] 0

[1] 21

[1] 0

In [15]:
## SAIGE-QTL SMR gene list
saige_df_sign = saige_df_common[saige_df_common$p_SMR < (0.05/length(common_combs)),]
saige_df_sign

probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,⋯,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_HEIDI,nsnp_HEIDI,Cell_type,comb,qv
<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<dbl>
HLA-DRB1,6,HLA-DRB1,32546546,rs74927567,6,32597064,G,A,0.394101,⋯,0.0392904,5.966581e-13,-1.05605,0.228554,3.826358e-06,0.005936424,20,NKact,HLA-DRB1_NKact,0.03921362
